In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from data_model import CyclingAnalyticsDataModel, PerfProDataModel
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# create the data model
db_file = r'D:\perfpro_data\history.db'
perfpro_file = r'D:\perfpro_data\RaceLog.mdb'
md = CyclingAnalyticsDataModel(db_file)
pp = PerfProDataModel(perfpro_file)

## Leitura dos dados

In [13]:
#user = 'Victor São Paulo  Ruela' 
user =  'Japao carvalho'

train_performances = pp.get_performance_by_user(user)

train_data = md.get_train_by_user(user)
train_data.drop_duplicates(inplace=True)

In [15]:
def calculate_ftp_zones(ftp, zone):
    """ Estimates the FTP zones:
        https://support.wattbike.com/hc/en-us/articles/115001848349-Functional-Threshold-Power-FTP-Training-Zones
    Args:
        ftp ([type]): [description]
    """
    zones_dict = {
        'Z1': 0.55*ftp,
        'Z2': 0.75*ftp,
        'Z3': 0.90*ftp,
        'Z4': 1.05*ftp,
        'Z5': 1.2*ftp,
        'Z6': 1.5*ftp,
    }
    return zones_dict[zone]

for i in range(6):
    zone = f'Z{i+1}'
    train_performances[zone] = train_performances['RunFTP'].apply(lambda x: calculate_ftp_zones(x, zone))

train_performances.tail()

,userName,dateStamp,userWeight,minutes,aveSpeed,aveWatts,aveHR,aveRPMs,NormPower,aveCalories,...,WorkoutType,aveLoad,MinLoad,MaxLoad,Z1,Z2,Z3,Z4,Z5,Z6
48,Japao carvalho,2021-01-27 08:06:00,65 kgs,60.0002,13.43,102.1,0.0,79.25,144.664660,593.775452,...,Intervals,122.1,71,214,4.125,5.625,6.75,7.875,9.0,11.25
49,Japao carvalho,2021-01-29 16:53:00,65 kgs,5.9086,4.48,31.2,0.0,42.61,95.407236,38.449162,...,Endurance,132.7,98,173,4.125,5.625,6.75,7.875,9.0,11.25
50,Japao carvalho,2021-02-03 08:12:00,65 kgs,55.0002,16.43,114.2,0.0,86.24,160.945407,605.678284,...,Indoor Ride,125.8,71,231,4.125,5.625,6.75,7.875,9.0,11.25
51,Japao carvalho,2021-02-03 14:14:00,65 kgs,0.3194,5.53,15.4,0.0,49.85,0.000000,0.363702,...,Indoor Ride,89.0,89,89,4.125,5.625,6.75,7.875,9.0,11.25
52,Japao carvalho,2021-02-06 09:41:00,65 kgs,24.1590,12.10,64.5,0.0,85.26,87.716868,144.733261,...,FTP Test,134.3,89,178,4.125,5.625,6.75,7.875,9.0,11.25


In [ ]:
train_data_processed = pd.pivot_table(train_data, values='value', columns='variable', index=['timestamp','train_id'])
train_data_processed.reset_index(inplace=True)
# creates the train elapsed time in minutes column
def calculate_duration(group):
    group['elapsed_minutes'] = (group['timestamp'] - group['timestamp'].iloc[0]).dt.total_seconds()/60.0
    return group
train_data_processed = train_data_processed.groupby('train_id').apply(calculate_duration)
# remove invalid columns
train_data_processed.drop(columns=['altitude','enhanced_altitude','grade','enhanced_speed'], inplace=True)
# remove records without speed data
def check_data_quality(group):
    is_good = True
    len_group = len(group['cadence'])
    # amount of null speed values
    speed_null_pct = 100.0*group['speed'].isnull().sum()/len_group
    is_good = is_good and speed_null_pct < 10

    # amount of null rpm values
    cadence_null_pct = 100.0*group['cadence'].isnull().sum()/len_group
    is_good = is_good and cadence_null_pct < 10

    # amount of null power values
    power_null_pct = 100.0*group['power'].isnull().sum()/len_group
    is_good = is_good and power_null_pct < 10


    # cadence is fixed for most of the time
    for var in ['cadence', 'speed', 'power']:
        chunks = np.array_split(group[var], 120)
        chunks_fixed = 100.0*np.sum([np.std(chunk) <= 0.1 for chunk in chunks]) / len(chunks)
        is_good = is_good and chunks_fixed < 10
        
    # cadence_fixed_pct = 100.0*sum(group['cadence'].rolling(10).std() <= 0.5)/len_group
    # is_good = is_good and cadence_fixed_pct < 10

    # # power is fixed for most of the time
    # power_fixed_pct = 100.0*sum(group['power'].rolling(10).std() <= 0.1)/len_group
    # is_good = is_good and power_fixed_pct < 80

    # # speed is fixed for most of the time
    # speed_fixed_pct = 100.0*sum(group['speed'].rolling(10).std() <= 0.1)/len_group
    # is_good = is_good and speed_fixed_pct < 80

    # minimum duration
    is_good = is_good and group['elapsed_minutes'].max() > 10
    
    return is_good

train_data_quality = train_data_processed.groupby('train_id').apply(check_data_quality).reset_index()
train_data_quality.columns = ['train_id', 'status']

valid_train_ids = train_data_quality.loc[train_data_quality.status == True, 'train_id']
print(f'% good trains: {100.0*len(valid_train_ids)/len(train_data_quality)}')

# remove bad quality training data
train_data_processed = train_data_processed.loc[train_data_processed.train_id.isin(valid_train_ids),:]
# interpolate missing values
numeric_columns = ['cadence','distance','heart_rate','power','speed','cadence256']
# train_data_processed[numeric_columns] = train_data_processed[numeric_columns].interpolate('linear')
# convert the speed from m/s to km/h
train_data_processed['speed'] = train_data_processed['speed'] * 3.6

# remove invalid power/cadence values
train_data_processed.loc[(train_data_processed.cadence <= 10.0)|(train_data_processed.power <= 10.0)|(train_data_processed.speed <= 2.0), ['cadence', 'power', 'speed']] = np.nan
#train_data_processed.loc[train_data_processed.power <= 10.0, 'power'] = np.nan
#train_data_processed.loc[train_data_processed.speed <= 5.0, 'speed'] = np.nan

# add seasonal features
train_data_processed['month'] = train_data_processed['timestamp'].dt.month
train_data_processed['week_of_year'] = train_data_processed['timestamp'].dt.week
train_data_processed['year'] = train_data_processed['timestamp'].dt.year

train_data_processed.head()

## Visualização

In [ ]:
def plot_train_values(train_data, train_id):
    data = train_data.loc[train_data.train_id == train_id, :]
    fig, ax = plt.subplots(figsize=(12,5))
    ax_sec = ax.twinx()
    ax.plot(data.elapsed_minutes, data.cadence, 'r', label='Cadência (rpm)')
    ax.plot(data.elapsed_minutes, data.speed, 'b', label='Velocidade (km/h)')
    ## add an empty line just for the legend
    ax.plot(data.elapsed_minutes, np.ones_like(data.power)*np.nan, 'k', label='Potência (W)')
    ax_sec.plot(data.elapsed_minutes, data.power, 'k', label='Potência (W)')
    ax.legend()
    ax.set_xlabel('Tempo (min)')
    ax.set_ylabel('Cadência, Velocidade')
    ax_sec.set_ylabel('Potência')
    ax.set_title(f'Treino: {train_id}')

In [ ]:
train_to_display = valid_train_ids.iloc[5]
for i in valid_train_ids:
    plot_train_values(train_data_processed, i)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16,8))
sns.scatterplot(data=train_data_processed, x='cadence', y='power', hue='week_of_year', ax=ax[0], palette='turbo')
sns.scatterplot(data=train_data_processed, x='cadence', y='speed', hue='week_of_year', ax=ax[1], palette='turbo')
sns.scatterplot(data=train_data_processed, x='power', y='speed', hue='week_of_year', ax=ax[2], palette='turbo')

### Feature Engineering

In [ ]:
def calculate_summaries(group):
    group['duration'] = group.elapsed_minutes.max()

    group['normalized_power'] = np.power(np.mean(np.power(group.power.rolling(30).mean(), 4)), 1/4)
    group['normalized_cadence'] = np.power(np.mean(np.power(group.cadence.rolling(30).mean(), 4)), 1/4)
    
    group['average_power'] = group.power.mean()
    group['average_speed'] = group.speed.mean()    
    group['average_cadence'] = group.cadence.mean()
    # You can estimate FTP with your best recent 20-minute power value 
    # (either from a dedicated 20-minute test or a sufficiently hard 20-minute effort from a race or workout). 
    # Multiply that value by 95% to get your FTP
    group['predicted_ftp'] = group.power.rolling(window=20*60).mean().max() * 0.95

    group['itensity_factor'] = group['normalized_power'] / group['predicted_ftp']
    group['stress_score'] = (group['duration']*60.0*group['itensity_factor']*group['itensity_factor'])/(group['normalized_power']*3600.0)*100.0

    return group.iloc[0][['duration', 'normalized_cadence','normalized_power','average_power','average_speed','average_cadence','predicted_ftp','itensity_factor','stress_score']]

train_data_summaries = train_data_processed.groupby('train_id').apply(calculate_summaries)
# train_data_summaries = pd.melt(train_data_summaries.reset_index(), id_vars=['train_id'], value_vars=train_data_summaries.columns)
train_data_summaries = train_data_summaries.join(train_data.groupby('train_id')[['date','name']].first(), how='inner')

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(train_data_summaries.date, train_data_summaries.predicted_ftp, 'ro', label='FTP')
#sns.pairplot(train_data_summaries)